# 01.Tensorflow Estimator of Tensorflow v1
__author__ = "Trung Phong Le"<br>
__email__ = "letrungphong95@gmail.com"<br>
__startDate__ = "19/02/2020"<br>
__version__ = "0.0.1"<br>
__status__ = "Being a Magical Dev"<br>
__maintainer__ = "Trung Phong Le"


Ref:<br> 
https://github.com/aymericdamien/TensorFlow-Examples<br>
https://towardsdatascience.com/first-contact-with-tensorflow-estimator-69a5e072998d<br>

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
print("Tensorflow version: ", tf.__version__)

Tensorflow version:  1.14.0


## 1.1 Loading MNIST dataset

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Reshape and normalize 
# x_train = x_train.reshape(x_train.shape[0], 28, 28, 1) / 255.0
# x_test = x_test.reshape(x_test.shape[0], 28, 28, 1) / 255.0
x_train = x_train.reshape(x_train.shape[0], 784) / 255.0
x_test = x_test.reshape(x_test.shape[0], 784) / 255.0
# # Split training data to train and validation data 
x_val = x_train[50000:]
y_val = y_train[50000:]
x_train = x_train[:50000]
y_train = y_train[:50000]
print("=======> Number of training images: ", x_train.shape[0], ".Shape: ", x_train.shape)
print("=======> Number of validation images: ", x_val.shape[0], ".Shape: ", x_val.shape)
print("=======> Number of testing images: ", x_test.shape[0], ".Shape: ", x_test.shape)

## 1.2 Building Tensorflow estimator model

In [3]:
def convolution_network(x, classes, is_reuse):
    """ TODO: Updating
    """
    with tf.variable_scope('ConvolutionNet', reuse=is_reuse):
        # Input 
        x = tf.reshape(x, shape=[-1, 28, 28, 1])
        # Convolution 1
        conv1 = tf.layers.conv2d(inputs=x, filters=32, kernel_size=(3, 3), activation=tf.nn.relu, 
                                 strides=(1, 1), padding='valid', name='C1')
        conv1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=(2, 2), strides=(1, 1),
                                        padding='valid', name='P1')
        # Flatten
        flatten = tf.contrib.layers.flatten(inputs=conv1)
        # Full connected
        fc1 = tf.layers.dense(inputs=flatten, units=128, activation=tf.nn.relu, name='F1')
        # Output softmax classification
        out = tf.layers.dense(inputs=fc1, units=classes)
    return out 

In [4]:
def model_fn(features, labels, mode):
    """ TODO: Updating
    """
    # Build Network
    pred_logits = convolution_network(x=features, classes=n_classes, is_reuse=False)
    # Predictions 
    pred_classes = tf.argmax(pred_logits, axis=1)
    pred_probas = tf.nn.softmax(pred_logits)
    # Check if train or test 
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
    # Define loss and optimizer for training 
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=pred_logits, labels=tf.cast(labels, dtype=tf.int32)
    ))
    tf.summary.scalar('loss', loss_op)
    # Evaluation accuracy
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes, name='acc')
    tf.summary.scalar('accuracy', acc_op[1])
    # Evaluation
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode=mode, 
            loss=loss_op,
            eval_metric_ops={'accuracy/eval': acc_op},
            evaluation_hooks=None,
        )
    # Optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    # Visualize in cmd
    train_hook_list = []
    train_tensors_log = {'accuracy': acc_op[1],
                         'loss': loss_op}
    train_hook_list.append(tf.train.LoggingTensorHook(
        tensors=train_tensors_log, every_n_iter=100))
    # Define estimator model
    estimator_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op = train_op,
        eval_metric_ops={'accuracy':acc_op},
        training_hooks=train_hook_list
    )
    return estimator_specs

model = tf.estimator.Estimator(model_fn, model_dir='checkpoint')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'checkpoint', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f25718dd978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## 1.3 Training model

In [7]:
batch_size = 32
num_steps = 2
n_classes = 10
learning_rate = 0.001
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x_train, y=y_train, batch_size=batch_size, shuffle=True
)
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x_val, y=y_val, batch_size=batch_size, shuffle=False
)

In [8]:
# Train Model
# model.train(train_input_fn, steps=num_steps)
for temporal_index in range(num_steps):
    model.train(input_fn=train_input_fn)
    result_eval = model.evaluate(input_fn=val_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into checkpoint/model.ckpt.
INFO:tensorflow:loss = 2.310034343482256, step = 1
INFO:tensorflow:accuracy = 0.0625, loss = 2.310034343482256
INFO:tensorflow:global_step/sec: 33.5238
INFO:tensorflow:loss = 0.21817331706586082, step = 101 (2.984 sec)
INFO:tensorflow:accuracy = 0.5, loss = 0.21817331706586082 (2.984 sec)
INFO:tensorflow:global_step/sec: 37.0087
INFO:tensorflow:loss = 0.22913961538588512, step = 201 (2.702 sec)
INFO:tensorflow:accuracy = 0.65625, loss = 0.22913961538588512 (2.702 sec)
INFO:tensorflow:global_step/sec: 36.7927
INFO:tensorflow:loss = 0.41517474364687135, step = 301 (2.718 sec)
INFO:tensorflow:accuracy = 0.7265625, loss = 0.41517474364687135 (2.718 sec)
INFO:tensorflow:global_step/sec: 35.

INFO:tensorflow:global_step/sec: 31.3596
INFO:tensorflow:loss = 0.07214597483727853, step = 1601 (3.189 sec)
INFO:tensorflow:accuracy = 0.9080882, loss = 0.07214597483727853 (3.186 sec)
INFO:tensorflow:global_step/sec: 33.2281
INFO:tensorflow:loss = 0.009610303089004726, step = 1701 (3.010 sec)
INFO:tensorflow:accuracy = 0.9131944, loss = 0.009610303089004726 (3.010 sec)
INFO:tensorflow:global_step/sec: 33.0812
INFO:tensorflow:loss = 0.03709095187749602, step = 1801 (3.023 sec)
INFO:tensorflow:accuracy = 0.9177632, loss = 0.03709095187749602 (3.022 sec)
INFO:tensorflow:Saving checkpoints for 1875 into checkpoint/model.ckpt.
INFO:tensorflow:Loss for final step: 0.11231381615223181.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-03-02T14:53:06Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from checkpoint/model.ckpt-1875
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running loc

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from checkpoint/model.ckpt-1875
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1875 into checkpoint/model.ckpt.
INFO:tensorflow:loss = 0.0034974407979126033, step = 1876
INFO:tensorflow:accuracy = 1.0, loss = 0.0034974407979126033
INFO:tensorflow:global_step/sec: 32.7215
INFO:tensorflow:loss = 0.00168683015487204, step = 1976 (3.057 sec)
INFO:tensorflow:accuracy = 1.0, loss = 0.00168683015487204 (3.056 sec)
INFO:tensorflow:global_step/sec: 32.7947
INFO:tensorflow:loss = 0.003557848934829048, step = 2076 (3.050 sec)
INFO:tensorflow:accuracy = 1.0, loss = 0.003557848934829048 (3.052 sec)
INFO:tensorflow:global_step/sec: 34.0586
INFO:tensorflow:loss = 0.0035046042225406306, step = 2176 (2.937

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-03-02T14:54:04Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from checkpoint/model.ckpt-3750
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-03-02-14:54:07
INFO:tensorflow:Saving dict for global step 3750: accuracy/eval = 0.9846, global_step = 3750, loss = 0.050776556
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3750: checkpoint/model.ckpt-3750


## 1.4 Predicting model

In [7]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x_test, y=y_test, batch_size=batch_size, shuffle=False
)

In [8]:
acc = model.evaluate(test_input_fn)
print('Accuracy: ', acc['accuracy'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-03-02T12:00:48Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/tmppom82lph/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-03-02-12:00:51
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.9781, global_step = 1000, loss = 0.06822193
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmp/tmppom82lph/model.ckpt-1000
Accuracy:  0.9781


## 1.5 Loading saved model